<a href="https://colab.research.google.com/github/IlyaZutler/GBR-agents/blob/main/GBR_Agents_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import matplotlib.pyplot as plt
!pip install simpy
import simpy
import random



In [26]:
np.log(20), np.random.lognormal(3, 0.5)

(2.995732273553991, 29.32014963064012)

In [29]:

class Agent:
    def __init__(self, env, location, speed):
        self.env = env  # Ссылка на объект окружения симуляции
        self.location = location
        self.speed = speed
        self.state = 'free'
        self.alarm = None
        self.start_time = None
        self.busy_time = 0

    def travel_time(self, target):
        return abs(self.location - target) / self.speed

    def handle_alarm(self, alarm):
        self.state = 'busy'
        self.alarm = alarm
        self.start_time = self.env.now
        #print(f"Agent starting to handle alarm at location {alarm.location} with complexity {alarm.complexity} at time {self.env.now}")

        # Время на дорогу до тревоги
        travel_to_alarm = self.travel_time(alarm.location)
        yield self.env.timeout(travel_to_alarm)

        # Время на обработку тревоги
        alarm.arrival_time = self.env.now
        print(f"Agent arrived at alarm at time {self.env.now}")
        yield self.env.timeout(alarm.complexity)

        # Агенты остаются занятыми до завершения обработки тревоги
        self.busy_time += self.env.now - self.start_time
        self.state = 'free'
        self.alarm = None
        print(f"Agent finished handling alarm at time {self.env.now}")

        # Время на возвращение к месту дислокации
        yield self.env.timeout(travel_to_alarm)

class Alarm:
    def __init__(self, location, complexity):
        self.location = location
        self.complexity = complexity
        self.arrival_time = None

class Simulation:
    def __init__(self, num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration):
        self.env = simpy.Environment()
        # Равномерное распределение агентов вдоль отрезка
        if num_agents == 1:
            self.agents = [Agent(self.env, segment_length / 2, agent_speed)]
        else:
            self.agents = [Agent(self.env, i * segment_length / (num_agents - 1), agent_speed) for i in range(num_agents)]

        self.segment_length = segment_length
        self.alarm_rate = alarm_rate
        self.complexity_mean = complexity_mean
        self.complexity_sigma = complexity_sigma
        self.threshold = threshold
        self.simulation_duration = simulation_duration
        self.alarms = []
        self.total_alarms = 0
        self.total_response_time = 0
        self.delayed_alarms = 0

    def generate_alarms(self):
        while True:
            yield self.env.timeout(random.expovariate(self.alarm_rate))
            location = random.uniform(0, self.segment_length)
            complexity = np.random.lognormal(self.complexity_mean, self.complexity_sigma)
            alarm = Alarm(location, complexity)
            self.alarms.append(alarm)
            self.total_alarms += 1
            print(f"Generated alarm at location {location} with complexity {complexity} at time {self.env.now}")
            self.env.process(self.dispatch_agent(alarm))


    def dispatch_agent(self, alarm):
        available_agents = [agent for agent in self.agents if agent.state == 'free']
        if available_agents:
            closest_agent = min(available_agents, key=lambda agent: agent.travel_time(alarm.location))
            closest_agent.start_time = self.env.now
            self.total_response_time += closest_agent.travel_time(alarm.location)
            if closest_agent.travel_time(alarm.location) > self.threshold:
                self.delayed_alarms += 1
            yield self.env.process(closest_agent.handle_alarm(alarm))
        else:
            print(f"No available agents for alarm at location {alarm.location} at time {self.env.now}")
            yield self.env.timeout(0)  # Ничего не делаем, если нет свободных агентов


    def run(self):
        self.env.process(self.generate_alarms())
        self.env.run(until=self.simulation_duration)

    def report(self):
        print(f"Simulation Duration: {self.simulation_duration}")
        print(f"Total Alarms: {self.total_alarms}")
        print(f"Average Response Time: {self.total_response_time / self.total_alarms if self.total_alarms else 0}")
        print(f"Fraction of Delayed Alarms: {self.delayed_alarms / self.total_alarms if self.total_alarms else 0}")
        for i, agent in enumerate(self.agents):
            busy_time_fraction = agent.busy_time / self.simulation_duration
            print(f"Agent {i} Busy Time Fraction: {busy_time_fraction}")

def visualize(simulation, interval=1):
    fig, ax = plt.subplots(figsize=(10, 1))
    ax.set_xlim(0, simulation.segment_length)
    ax.set_ylim(-1, 1)

    while simulation.env.peek() < float('inf'):
        ax.clear()
        ax.set_xlim(0, simulation.segment_length)
        ax.set_ylim(-1, 1)

        for agent in simulation.agents:
            if agent.state == 'free':
                ax.plot(agent.location, 0, 'bo')
            elif agent.state == 'busy':
                if agent.alarm is not None:
                    ax.plot(agent.alarm.location, 0, 'mo')
                ax.plot(agent.location, 0, 'go')

        for alarm in simulation.alarms:
            if alarm.arrival_time is None:
                ax.plot(alarm.location, 0, 'ro')

        plt.pause(interval)
        simulation.env.step()

    plt.show()

# Параметры моделирования
num_agents = 5
segment_length = 100
agent_speed = 30/60 # Cкорость
alarm_rate = 50/(24*60)
complexity_mean = np.log(20)  # Cложность 20 мин
complexity_sigma = 0.5
threshold = 10
simulation_duration = 24*60  # Задаем время моделирования в минутах (24 часа)

# Создание и запуск симуляции
sim = Simulation(num_agents, segment_length, agent_speed, alarm_rate, complexity_mean, complexity_sigma, threshold, simulation_duration)
sim.run()

# Отчёт о результатах
sim.report()

# Визуализация процесса
#visualize(sim, interval=0.1)

Generated alarm at location 11.48192618219337 with complexity 11.689313814269015 at time 11.425692772629057
Generated alarm at location 29.10282064962495 with complexity 28.82330857493076 at time 20.10697064695124
Agent arrived at alarm at time 28.312611946201137
Agent arrived at alarm at time 34.389545137015794
Generated alarm at location 56.62441343224894 with complexity 42.25704156204745 at time 36.154463379898274
Agent finished handling alarm at time 46.07885895128481
Agent arrived at alarm at time 49.40329024439616
Generated alarm at location 47.564142143929566 with complexity 26.589556418463047 at time 49.417479037804
Agent finished handling alarm at time 57.135920521131894
Generated alarm at location 86.98185783725117 with complexity 20.816861892928568 at time 78.73331504219621
Generated alarm at location 88.69052165498792 with complexity 24.3447762876046 at time 83.48002599983964
Agent finished handling alarm at time 91.66033180644361
Agent arrived at alarm at time 104.28919474